<a href="https://colab.research.google.com/github/Ami03sa/Openai_projects-/blob/pdf_rag_system/Speak_with_any_PDF_file_PDF_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pdf ai machine using RAG


# 2. Libraries import

In [46]:
pip install openai

In [44]:
pip install PyPDF2


In [45]:
pip install pinecone

In [1]:
import os
import openai
import PyPDF2
import random
import pinecone

from openai import OpenAI

# 3. Working with PDF files

![](https://miro.medium.com/v2/resize:fit:1400/1*FWwgOvUE660a04zoQplS7A.png)



### 3.1 Setting up API Key

In [43]:
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

### 3.2 Loading a PDF file




In [6]:
# Function to load a random PDF from a given directory
def load_pdf(file_name):
    # Read the PDF file
    pdf_file = open(file_name, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text_content = ""
    # Extract text from each page
    for page in range(len(pdf_reader.pages)):
        text_content += pdf_reader.pages[page].extract_text()

    pdf_file.close()

    return text_content

In [7]:
# Function to chunk text by number of words or characters with a given size and overlap
def chunk_text(text, chunk_size=1500, chunk_overlap=100, by='word'):
    if by not in ['word', 'char']:
        raise ValueError("Invalid value for 'by'. Use 'word' or 'char'.")

    chunks = []
    if by == 'word':
        text = text.split()
    elif by == 'char':
        text = text

    current_chunk_start = 0
    while current_chunk_start < len(text):
        current_chunk_end = current_chunk_start + chunk_size
        if by == 'word':
            chunk = " ".join(text[current_chunk_start:current_chunk_end])
        else:
            chunk = text[current_chunk_start:current_chunk_end]

        chunks.append(chunk)
        current_chunk_start += (chunk_size - chunk_overlap)


    return chunks

In [8]:
pdf_loaded = load_pdf("state_of_ai_docs.pdf")

In [9]:
chunks = chunk_text(pdf_loaded, by='char')

In [10]:
len(chunks)

28

## 4. Building RAG system (Retrieval Augmented System)

In [33]:
# Pinecone init

from pinecone import Pinecone

pc = Pinecone(api_key="")
index = pc.Index("rag")

In [34]:
for i in range(len(chunks)):
    vector = client.embeddings.create(
        model="text-embedding-ada-002",
        input=chunks[i])
    vector = vector.data[0].embedding

    upsert_response = index.upsert(
    vectors=[
        (
         str(i),
         vector,
         {"chunk_content": chunks[i]}
        )
    ])

### 5. Building an interface to get proper answer based on the documentation


In [35]:
# Ask user a question
user_request = input("Ask some question regarding the document: ")

# Create embedding for the question
user_vector = client.embeddings.create(
    model="text-embedding-ada-002",
    input=user_request
)

# Extract the embedding vector
user_vector = user_vector.data[0].embedding

# Query Pinecone index with keyword arguments
matches = index.query(
    vector=user_vector,
    top_k=1,
    include_metadata=True
)








Ask some question regarding the document: top ai trends in 2023/2024


In [42]:
messages = [
    {
        "role": "system",
        "content": """I want you to act as a support agent. Your name is "My Super Assistant".
        You will provide me with answers from the given info below.
        If the answer is not included, say exactly "I do not have information about this." and stop after that and not not answer anything or generate any text.
        If you can find answer to the question, say "This is what i can find" and provide the information found.
        Refuse to answer any question not about the info. Never break character."""
    },
    {
        "role": "system",
        "content": f"Here is the info:\n{matches['matches'][0]['metadata']['chunk_content']}"
    },
    {
        "role": "user",
        "content": user_request
    }
]

chat_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    temperature=0,
    max_tokens=400,
)


assistant_reply = chat_response.choices[0].message.content
#print("Assistant:", assistant_reply)
print()
print("Context: ", matches['matches'][0]['metadata']['chunk_content'])



Context:  As organizations rapidly deploy generative AI tools, survey respondents 
expect significant effects on their industries and workforces.The state of AI in 
2023: Generative AI’s 
breakout year
August 2023The state of AI in 2023: Generative AI’s breakout yearThe latest annual McKinsey Global Survey  on the current  
state of AI confirms the explosive growth of generative AI  
(gen AI) tools. Less than a year after many of these tools debuted, 
one-third of our survey respondents say their organizations are 
using gen AI regularly in at least one business function. Amid 
recent advances, AI has risen from a topic relegated to tech 
employees to a focus of company leaders: nearly one-quarter  
of surveyed C-suite executives say they are personally using  
gen AI tools for work, and more than one-quarter of respondents 
from companies using AI say gen AI is already on their boards’ 
agendas. What’s more, 40 percent of respondents say their 
organizations will increase their inves